#necessary libraries: TensorFlow, NumPy, Matplotlib, scikit-learn, Pillow, and pandas

In [ ]:
!pip install tensorflow numpy matplotlib scikit-learn pillow pandas

#imports the os module for interacting with the operating system and ImageDataGenerator from TensorFlow Keras for augmenting image data for training the model

In [27]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Defines the directories where the training and testing images are stored.

In [28]:
train_dir = 'images/training'
test_dir = 'images/testing'

#creates an ImageDataGenerator object for the training data with various augmentations such as rescaling, rotation, shifting, shearing, zooming, and flipping.

In [29]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

#creates an ImageDataGenerator object for the test data, rescaling the pixel values to the range [0, 1].

In [30]:
test_datagen = ImageDataGenerator(rescale=1./255)

#creates a data generator for the training images, specifying the directory, target size, batch size, and class mode

In [31]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 2367 images belonging to 3 classes.


a data generator for the test images, specifying the directory, target size, batch size, and class mode.

In [32]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 591 images belonging to 3 classes.


imports necessary classes from TensorFlow Keras to build a sequential model with convolutional, pooling, flattening, dense, and dropout layers

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

#def create_baseline_model: Defines a new function named create_baseline_model.
#input_shape=(224, 224, 3): The function takes an optional parameter input_shape, which defaults to (224, 224, 3). This represents the shape of the input images (224x224 pixels with 3 color channels - RGB).
#Sequential(): Creates a new Sequential model, which is a linear stack of layers.
#Conv2D(32, (3, 3), activation='relu', input_shape=input_shape): Adds a 2D convolutional layer with 32 filters, a 3x3 kernel, ReLU activation, and the specified input shape. This layer detects features in the input image.
#MaxPooling2D((2, 2)): Adds a max-pooling layer with a 2x2 pool size. This reduces the spatial dimensions of the feature maps, which helps in reducing computation and controlling overfitting.
#Conv2D(64, (3, 3), activation='relu'): Adds another 2D convolutional layer with 64 filters and ReLU activation.
#MaxPooling2D((2, 2)): Adds another max-pooling layer with a 2x2 pool size.
#Conv2D(128, (3, 3), activation='relu'): Adds a third 2D convolutional layer with 128 filters and ReLU activation.
#MaxPooling2D((2, 2)): Adds another max-pooling layer with a 2x2 pool size.
#Flatten(): Flattens the 3D feature maps to a 1D vector, preparing it for the dense (fully connected) layers.
#Dense(128, activation='relu'): Adds a dense layer with 128 units and ReLU activation. This layer learns high-level features.
#Dropout(0.5): Adds a dropout layer with a 50% dropout rate. This helps in reducing overfitting by randomly setting half of the inputs to zero during training.
#Dense(1, activation='sigmoid'): Adds a dense layer with 1 unit and sigmoid activation. This is the output layer for binary classification, outputting a probability between 0 and 1.
#optimizer='adam': Uses the Adam optimizer, which is an adaptive learning rate optimization algorithm.
loss='binary_crossentropy': Uses binary cross-entropy as the loss function, which is appropriate for binary classification tasks.
#metrics=['accuracy']: Specifies that accuracy should be tracked as a metric during training.
Returns the compiled model.
#create_baseline_model(): Calls the function to create the baseline model.
#baseline_model.summary(): Prints a summary of the model architecture, showing the layers, output shapes, and the number of parameters.

In [34]:
def create_baseline_model(input_shape=(224, 224, 3)):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

baseline_model = create_baseline_model()
baseline_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 54, 54, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 26, 26, 128)      

trains the baseline model using the training data generator and validates it using the test data generator for 3 epochs.

In [35]:
history_baseline = baseline_model.fit(train_generator, validation_data=test_generator, epochs=3)

Epoch 1/3
74/74 [==============================] - 280s 4s/step - loss: -14898040.0000 - accuracy: 0.4795 - val_loss: -116103712.0000 - val_accuracy: 0.4873
Epoch 2/3
74/74 [==============================] - 280s 4s/step - loss: -2652317440.0000 - accuracy: 0.4808 - val_loss: -10312916992.0000 - val_accuracy: 0.4873
Epoch 3/3
74/74 [==============================] - 304s 4s/step - loss: -50339033088.0000 - accuracy: 0.4808 - val_loss: -133815517184.0000 - val_accuracy: 0.4873


 imports the numpy library, which is used for numerical operations. It then creates ensemble predictions by averaging the predictions from the baseline, ResNet50, and EfficientNet models and rounding them to the nearest integer.

In [37]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

This function creates a fine-tuned ResNet50 model for binary classification. It removes the top layers of the pre-trained ResNet50, adds global average pooling, dense, and dropout layers, and compiles the model with the Adam optimizer and binary cross-entropy loss. The base ResNet50 layers are frozen to prevent retraining.

In [38]:
def create_resnet50_model(input_shape=(224, 224, 3)):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

resnet50_model = create_resnet50_model()
resnet50_model.summary()

94765736/94765736 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                         

This block trains the fine-tuned ResNet50 model using the training data generator and validates it using the test data generator for 3 epochs.

In [39]:
history_resnet50 = resnet50_model.fit(train_generator, validation_data=test_generator, epochs=3)

Epoch 1/3
74/74 [==============================] - 470s 6s/step - loss: -32.8188 - accuracy: 0.4778 - val_loss: -78.0321 - val_accuracy: 0.4873
Epoch 2/3
74/74 [==============================] - 473s 6s/step - loss: -159.7166 - accuracy: 0.4808 - val_loss: -257.5944 - val_accuracy: 0.4873
Epoch 3/3
74/74 [==============================] - 470s 6s/step - loss: -400.7887 - accuracy: 0.4808 - val_loss: -557.3835 - val_accuracy: 0.4873


This block imports ReduceLROnPlateau and EarlyStopping callbacks from TensorFlow Keras. These callbacks help in adjusting the learning rate and stopping the training early if the performance does not improve.
def fine_tune_resnet50_model(model, unfreeze_layers=10):: Defines a function named fine_tune_resnet50_model that takes a model and the number of layers to unfreeze as inputs.
for layer in model.layers[-unfreeze_layers:]: layer.trainable = True: Unfreezes the last unfreeze_layers layers of the model, allowing them to be trained.
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']): Compiles the model with the Adam optimizer, binary cross-entropy loss function, and accuracy metric.
return model: Returns the fine-tuned model.


In [44]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Fine-tuning ResNet-50 model
def fine_tune_resnet50_model(model, unfreeze_layers=10):
    # Unfreeze the last `unfreeze_layers` layers of the base model
    for layer in model.layers[-unfreeze_layers:]:
        layer.trainable = True

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

fine_tuned_resnet50_model = fine_tune_resnet50_model(resnet50_model)

ReduceLROnPlateau Callback

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6):
monitor='val_loss': This parameter specifies that the callback should monitor the validation loss during training.
factor=0.2: When triggered, the learning rate will be reduced by a factor of 0.2.
patience=3: The callback will wait for 3 epochs with no improvement in the monitored quantity (validation loss) before reducing the learning rate.
min_lr=1e-6: The learning rate will not be reduced below this minimum threshold.
EarlyStopping Callback

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True):
monitor='val_loss': This parameter specifies that the callback should monitor the validation loss during training.
patience=5: The callback will wait for 5 epochs with no improvement in the monitored quantity (validation loss) before stopping the training.
restore_best_weights=True: Once training is stopped, the model weights will be restored to the state of the best epoch (i.e., the epoch with the lowest validation loss).


In [45]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

1. Training the Model

history_fine_tuned_resnet50: This variable stores the history of the training process, including loss and accuracy metrics for each epoch.
fine_tuned_resnet50_model.fit(): This method trains the fine-tuned ResNet-50 model.
2. Parameters

train_generator: This parameter is the training data generator that provides batches of augmented training images.
validation_data=test_generator: This parameter specifies the validation data generator that provides batches of augmented validation images.
epochs=5: This parameter sets the number of epochs to 5, meaning the model will be trained for 5 complete passes over the entire training dataset.
callbacks=[reduce_lr, early_stopping]: This parameter specifies a list of callbacks to be used during training. In this case, the callbacks are reduce_lr and early_stopping.
3. Callbacks

ReduceLROnPlateau: This callback monitors the validation loss and reduces the learning rate by a factor of 0.2 if there is no improvement for 3 consecutive epochs, with a minimum learning rate of 0.1×10 ^−6
EarlyStopping: This callback monitors the validation loss and stops the training if there is no improvement for 5 consecutive epochs. It also restores the model weights to those of the epoch with the best validation loss.

In [46]:
history_fine_tuned_resnet50 = fine_tuned_resnet50_model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=5,
    callbacks=[reduce_lr, early_stopping]
)

Epoch 1/5
74/74 [==============================] - 472s 6s/step - loss: -1473.1949 - accuracy: 0.4808 - val_loss: -1346.3750 - val_accuracy: 0.4873 - lr: 0.0010
Epoch 2/5
74/74 [==============================] - 473s 6s/step - loss: -2872.3188 - accuracy: 0.4808 - val_loss: -1919.8953 - val_accuracy: 0.4873 - lr: 0.0010
Epoch 3/5
74/74 [==============================] - 523s 7s/step - loss: -5120.5146 - accuracy: 0.4808 - val_loss: -4106.7607 - val_accuracy: 0.4873 - lr: 0.0010
Epoch 4/5
74/74 [==============================] - 474s 6s/step - loss: -8677.0537 - accuracy: 0.4808 - val_loss: -8183.1777 - val_accuracy: 0.4873 - lr: 0.0010
Epoch 5/5
74/74 [==============================] - 495s 7s/step - loss: -14018.0137 - accuracy: 0.4808 - val_loss: -17359.7539 - val_accuracy: 0.4873 - lr: 0.0010


from tensorflow.keras.applications import EfficientNetB0: This line imports the EfficientNetB0 class from the tensorflow.keras.applications module. EfficientNetB0 is a pre-trained convolutional neural network model available in TensorFlow Keras that is designed for image classification tasks.

In [47]:
from tensorflow.keras.applications import EfficientNetB0

Function Definition

def create_efficientnet_model(input_shape=(224, 224, 3)):: Defines a function named create_efficientnet_model that takes an optional parameter input_shape, which defaults to (224, 224, 3). This represents the shape of the input images (224x224 pixels with 3 color channels - RGB).
Base Model

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape): Loads the EfficientNetB0 model with weights pre-trained on ImageNet and removes the top layers (fully connected layers). This allows for customization of the classification layers.
Adding Custom Layers

x = base_model.output: Retrieves the output of the base EfficientNetB0 model.
x = GlobalAveragePooling2D()(x): Adds a global average pooling layer to reduce the spatial dimensions of the feature maps.
x = Dense(128, activation='relu')(x): Adds a dense layer with 128 units and ReLU activation to learn complex features.
x = Dropout(0.5)(x): Adds a dropout layer with a 50% dropout rate to prevent overfitting.
predictions = Dense(1, activation='sigmoid')(x): Adds a dense layer with 1 unit and sigmoid activation for binary classification.
Model Compilation

model = Model(inputs=base_model.input, outputs=predictions): Creates a new model that includes both the base EfficientNetB0 layers and the custom classification layers.
for layer in base_model.layers: layer.trainable = False: Freezes the base EfficientNetB0 layers to prevent them from being retrained during the initial training phase.
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']): Compiles the model with the Adam optimizer, binary cross-entropy loss function, and accuracy metric.
Model Summary

efficientnet_model.summary(): Prints a summary of the model architecture, showing the layers, output shapes, and the number of parameters.

In [48]:
def create_efficientnet_model(input_shape=(224, 224, 3)):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

efficientnet_model = create_efficientnet_model()
efficientnet_model.summary()

16705208/16705208 [==============================] - 0s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_2[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)   

In [49]:
history_efficientnet = efficientnet_model.fit(train_generator, validation_data=test_generator, epochs=3)

Epoch 1/3
74/74 [==============================] - 277s 4s/step - loss: -28.9342 - accuracy: 0.4782 - val_loss: -74.2481 - val_accuracy: 0.4873
Epoch 2/3
74/74 [==============================] - 260s 4s/step - loss: -156.1465 - accuracy: 0.4808 - val_loss: -265.5513 - val_accuracy: 0.4873
Epoch 3/3
74/74 [==============================] - 303s 4s/step - loss: -411.2669 - accuracy: 0.4808 - val_loss: -588.5508 - val_accuracy: 0.4873


In [53]:
from sklearn.metrics import f1_score

import Accuracy metrics
Function Definition:
def get_predictions(model, data_generator):: Defines a function named get_predictions that takes two parameters:
model: The trained machine learning model.
data_generator: The data generator providing the input data for predictions.
Making Predictions

predictions = model.predict(data_generator): Uses the model to predict the output for the data provided by the data_generator. The predict method returns an array of probabilities.
Thresholding Predictions

return (predictions > 0.5).astype(int): Converts the predicted probabilities into binary class labels. If the probability is greater than 0.5, the prediction is classified as 1 (positive class); otherwise, it is classified as 0 (negative class). The astype(int) method ensures that the resulting array contains integer values.


In [54]:
def get_predictions(model, data_generator):
    predictions = model.predict(data_generator)
    return (predictions > 0.5).astype(int)

In [55]:
baseline_preds = get_predictions(baseline_model, test_generator)
resnet50_preds = get_predictions(fine_tuned_resnet50_model, test_generator)
efficientnet_preds = get_predictions(efficientnet_model, test_generator)

19/19 [==============================] - 51s 3s/step


Creating Ensemble Predictions

ensemble_preds = np.round((baseline_preds + resnet50_preds + efficientnet_preds) / 3):
baseline_preds, resnet50_preds, efficientnet_preds: These variables store the binary predictions (0 or 1) from the baseline model, the fine-tuned ResNet50 model, and the EfficientNet model, respectively.
(baseline_preds + resnet50_preds + efficientnet_preds) / 3: This expression calculates the average of the predictions from the three models. Since the predictions are binary (0 or 1), this average will be a value between 0 and 1.
np.round(...): The np.round function rounds the averaged predictions to the nearest integer (0 or 1). This step converts the averaged values back to binary predictions.

In [57]:
import numpy as np
ensemble_preds = np.round((baseline_preds + resnet50_preds + efficientnet_preds) / 3)

Retrieving Test Labels

test_labels = test_generator.classes: This line retrieves the true class labels for the test dataset from the test_generator. The classes attribute of the data generator returns an array of integer class labels corresponding to the images in the test set.
Calculating the F1 Score

f1 = f1_score(test_labels, ensemble_preds): This line calculates the F1 score for the ensemble model's predictions. The f1_score function from scikit-learn is used for this purpose. It takes two arguments:
test_labels: The true class labels for the test dataset.
ensemble_preds: The predicted class labels from the ensemble model.
The F1 score is a measure of a model's accuracy on a dataset and is the harmonic mean of precision and recall. It is especially useful for imbalanced datasets.

Printing the F1 Score

print(f'Ensemble Model F1 Score: {f1}'): This line prints the F1 score of the ensemble model, formatted as a string. The f before the string denotes an f-string, which allows for embedding expressions inside string literals.

In [58]:
test_labels = test_generator.classes
f1 = f1_score(test_labels, ensemble_preds)
print(f'Ensemble Model F1 Score: {f1}')

Ensemble Model F1 Score: 0.6552901023890785
